# Armageddon
## A simulation software produced by Popigai-2021

This is a simulation software for events of one spherical asteroid entering Earth atmosphere, it predicts damage zones, caused by the coming asteroid, on Earth surface. The software also produces some graphs on the interested __

### Usage guideline

The following cell contains all the basic parameters needed to start a simulation.

- After keys in all the parameters in the cell, select 'Cell' -> 'Run Cells' in the tool bar to start a parameter check; proceed to next step if the cell outputs 'OK to proceed'; detail report is output in case of failing the parameter check

- Select 'Kernel' -> 'Restart & Run All' in the tool bar would start the simulation, then scroll down to view the outputs; some cells would take some times to produce outputs depend on the parameters given

In [1]:
## Parameter for Earth
# This set the atmospheric density profile in the simulation
#     'exponential' is default; 'tabular' gives a better simulation
atmos_func = 'exponential'

## Parameters for asteroid (at altitude is 100,000 meters)
# This is the radius of the incoming asteroid (meter)
radius = 10
# This is the incoming angle of the asteroid (degree)
angle = 20
# This is the incoming velocity of the asteroid (m/s)
velocity = 19e3
# This is the density of the asteroid (kg/m^3)
density = 3000
# This is the strength of the asteroid (Pa)
strength = 1e6
# This is the entry latitude, longitude and bearing of the asteroid (degree)
entry_latitude, entry_longitude, bearing = 51.2, 0.7, -35.0

## Parameters for damage levels
# This set which level of damage is shown on a map
#     True indicates the level is shown on the map
levels = {
    'level 1': True,
    'level 2': True,
    'level 3': True,
    'level 4': True
}

# ================== below are parameter checks ==================
varis = [radius, angle, velocity, density,
         strength, entry_latitude, entry_longitude, bearing]
names = ['radius', 'angle', 'velocity', 'density',
         'strength', 'entry_latitude', 'entry_longitude', 'bearing']
types = [0, 1, 0, 0, 0, 2, 3, 4]

if atmos_func not in ['exponential', 'tabular']:
    raise ValueError('atmos_func must be \'exponential\' or \'tabular\'')
    
if (type(levels) != dict
    or len(levels) != 4
    or list(levels.keys()) != ['level 1', 'level 2', 'level 3', 'level 4']):
    raise ValueError('only changing value is allowed for levels')
for l in levels.values():
    if type(l) != bool: raise ValueError('only boolean is allowed in levels')
    

def check(var, var_name, var_type):
    if type(var) not in [int, float]: raise ValueError('%s must be a number' % var_name)
        
    min_d, max_d = 0, 0
    if var_type == 1: min_d, max_d = 0, 90
    elif var_type == 2: min_d, max_d = -90, 90
    elif var_type == 3: min_d, max_d = -180, 180
    elif var_type == 4: min_d, max_d = -180, 360

    if var_type == 0:
        if var <= 0: raise ValueError('%s must be a positive number' % var_name)  
    elif var < min_d or var > max_d:
        raise ValueError('%s must be a number between %i and %i' % (var_name, min_d, max_d))

for v, n, t in zip(varis, names, types):
    check(v, n, t)
    
print('OK to proceed')

OK to proceed


# === DO NOT CHANGE ANYTHING IN ANY CELL BELOW ===

In [2]:
import folium
import armageddon

# Initial value, You should use results from previous modules as the value of these parameters
burst_lat = entry_latitude
burst_lon = entry_longitude
blast_lat = 51.716033
blast_lon = -3.845419
radius_list = [0.3e3, 0.5e3]

# The PostcodeLocator tool
locator = armageddon.PostcodeLocator()

# Unit
postcodes = locator.get_postcodes_by_radius((blast_lat, blast_lon), radius_list)   
population = locator.get_population_of_postcode(postcodes)
armageddon.plot_results(burst_lat, burst_lon, blast_lat, blast_lon, radius_list, postcodes, population)


In [3]:
# Alternatively find the postcode sectors in the damage radii, and populations of the sectors
sectors = locator.get_postcodes_by_radius((blast_lat, blast_lon), radius_list, sector=True)
# print(sectors)

# for sector, the results from get_postcodes should be added one space for each postcodes before passing to get_population
for i in range(len(sectors)):
    for j in range(len(sectors[i])):
        sectors[i][j] = sectors[i][j][:4] + ' ' + sectors[i][j][4:]

population_sector = locator.get_population_of_postcode(sectors, sector=True)
# print(population_sector)
armageddon.plot_results(burst_lat, burst_lon, blast_lat, blast_lon, radius_list, sectors, population_sector, sector = True)